<a href="https://www.kaggle.com/code/mubashirjawad01/raw-fatal-pcg-mother-and-child-dataset?scriptVersionId=298260624" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [8]:
import os
import glob
import scipy.io
import matplotlib.pyplot as plt
import numpy as np

# **Data Load**

In [10]:
folder_path = '/kaggle/input/datasets/mubashirjawad01/raw-fetal-pcg-contaminated-with-mothers-pcg' 

#searching for sub folder of.mat  when recursive = true  
search_pattern = os.path.join(folder_path, '**', '*.mat')
mat_files = glob.glob(search_pattern, recursive=True)

#data save dictonary 
all_data = {}

print(f"Total .mat files found: {len(mat_files)}\n")

#loading the file
if len(mat_files) > 0:
    for file_path in mat_files:
        file_name = os.path.basename(file_path)
        try:
            all_data[file_name] = scipy.io.loadmat(file_path)
            print(f" Loaded: {file_name}")
        except Exception as e:
            print(f"❌ Error loading {file_name}: {e}")
            
    print("\n All files are loaded succesfully ")
else:
    print("❌ didn't Find Any Folder")

Total .mat files found: 21

 Loaded: B2_P4_S1.mat
 Loaded: B1_P3_S2.mat
 Loaded: B2_P4_S3.mat
 Loaded: B1_P3_S1.mat
 Loaded: B2_P5_S1.mat
 Loaded: B2_P1_S3.mat
 Loaded: B2_P3_S1.mat
 Loaded: B2_P3_S2.mat
 Loaded: B2_P2_S2.mat
 Loaded: B2_P3_S3.mat
 Loaded: B2_P4_S2.mat
 Loaded: B2_P1_S2.mat
 Loaded: B1_P2_S1.mat
 Loaded: B2_P5_S2.mat
 Loaded: B2_P2_S1.mat
 Loaded: B2_P2_S3.mat
 Loaded: B1_P2_S2.mat
 Loaded: B2_P1_S1.mat
 Loaded: B2_P5_S3.mat
 Loaded: B1_P1_S2.mat
 Loaded: B1_P1_S1.mat

 All files are loaded succesfully 


In [ ]:
import os
import glob
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import signal

# ১. পাথ সেটআপ (আপনার ফাইলগুলো যেখানে আছে)
folder_path = '/kaggle/input/your-dataset-name' # এখানে আপনার পাথ দিন
output_dir = 'spectrogram_images'
os.makedirs(output_dir, exist_ok=True)

mat_files = glob.glob(os.path.join(folder_path, '*.mat'))

# ইমেজ ও স্ট্যাটিস্টিকস স্টোর করার লিস্ট
image_stats = []

def get_class(filename):
    # ফাইলের নামের প্রথম অংশ (যেমন B1 বা B2) কে ক্লাস ধরছি
    return filename.split('_')[0]

print("Processing files and generating EDA...")

for file_path in mat_files:
    file_name = os.path.basename(file_path)
    label = get_class(file_name)
    
    # .mat ফাইল লোড
    data = scipy.io.loadmat(file_path)
    fs = data['Fs'].item()
    y = data['y'].flatten()
    
    # ২. সিগন্যালকে স্পেকট্রোগ্রামে রূপান্তর (ইমেজ তৈরি)
    f, t, Sxx = signal.spectrogram(y[:int(fs*5)], fs) # প্রথম ৫ সেকেন্ডের ডেটা
    
    plt.figure(figsize=(5, 5))
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
    plt.axis('off')
    img_path = os.path.join(output_dir, file_name.replace('.mat', '.png'))
    plt.savefig(img_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # ৩. ইমেজ EDA শুরু
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Mean and Std (Per image)
    mean_rgb = np.mean(img_rgb, axis=(0,1))
    std_rgb = np.std(img_rgb, axis=(0,1))
    
    # Brightness and Contrast
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    contrast = np.std(gray)
    
    # Saturation Clipping (Saturation max value 255 হলে তা ক্লিপড)
    s_channel = img_hsv[:,:,1]
    sat_clipping = np.sum(s_channel == 255) / s_channel.size
    
    image_stats.append({
        'file': file_name,
        'class': label,
        'mean_rgb': mean_rgb,
        'std_rgb': std_rgb,
        'brightness': brightness,
        'contrast': contrast,
        'sat_clipping': sat_clipping,
        'img_rgb': img_rgb,
        'img_hsv': img_hsv
    })

print("EDA Analysis Complete!")